# Instructor

[Instructor](https://python.useinstructor.com/) is a lightweight library that makes it easy to get structured data like JSON from LLMs.

In [ ]:
!pip install -U instructor openai weave -qqq

In order to use OpenAI with Instructor, you need to set the `OPENAI_API_KEY` environment variable. You can get your API key from [platform.openai.com/api-keys](https://platform.openai.com/api-keys).

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")

# Tracing

It’s important to store traces of language model applications in a central location, both during development and in production. These traces can be useful for debugging, and as a dataset that will help you improve your application.

Weave will automatically capture traces for [Instructor](https://python.useinstructor.com/) and [OpenaAI](https://platform.openai.com/docs/libraries/python-library). To start tracking, calling `weave.init()` and use the library as normal.

In [ ]:
import instructor
import weave
from pydantic import BaseModel
from openai import OpenAI


# Define your desired output structure
class UserInfo(BaseModel):
    user_name: str
    age: int

# Initialize Weave
weave.init(project_name="quickstart-instructor")

# Patch the OpenAI client
client = instructor.from_openai(OpenAI())

# Extract structured data from natural language
user_info = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=UserInfo,
    messages=[{"role": "user", "content": "John Doe is 30 years old."}],
)

| [![](https://weave-docs.wandb.ai/assets/images/instructor_lm_trace-25522be59f6e456773b176ce0b23b2f6.gif)](https://wandb.ai/geekyrakshit/quickstart-instructor/weave/calls) |
| --- |
| Weave will now track and log all LLM calls made using Instructor. You can view the traces in the Weave web interface. |

## Track your own ops

Wrapping a function with [`@weave.op`](https://weave-docs.wandb.ai/guides/tracking/ops) starts capturing inputs, outputs and app logic so you can debug how data flows through your app. You can deeply nest ops and build a tree of functions that you want to track. This also starts automatically versioning code as you experiment to capture ad-hoc details that haven't been committed to git.

Simply create a function decorated with [`@weave.op`](https://weave-docs.wandb.ai/guides/tracking/ops).

In the example below, we have the function `extract_person` which is the metric function wrapped with [`@weave.op`](https://weave-docs.wandb.ai/guides/tracking/ops). This helps us see how intermediate steps, such as OpenAI chat completion call.

In [ ]:
# Define your desired output structure
class Person(BaseModel):
    person_name: str
    age: int


# Extract structured data from natural language
@weave.op()
def extract_person(text: str) -> Person:
    return client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": text},
        ],
        response_model=Person,
    )


person = extract_person("My name is John and I am 20 years old")

| [![](https://weave-docs.wandb.ai/assets/images/instructor_op_trace-bcceed1c88e0610f8675d059e22e8b0a.png)](https://wandb.ai/geekyrakshit/quickstart-instructor/weave/calls) |
| --- |
| Decorating the extract_person function with [`@weave.op`](https://weave-docs.wandb.ai/guides/tracking/ops) traces its inputs, outputs, and all internal LM calls made inside the function. Weave also automatically tracks and versions the structured objects generated by [Instructor](https://python.useinstructor.com/). |

## Create a [`Model`](https://weave-docs.wandb.ai/guides/core-types/models) for easier experimentation

Organizing experimentation is difficult when there are many moving pieces. By using the [`Model`](https://weave-docs.wandb.ai/guides/core-types/models) class, you can capture and organize the experimental details of your app like your system prompt or the model you're using. This helps organize and compare different iterations of your app.

In addition to versioning code and capturing inputs/outputs, a [`Model`](https://weave-docs.wandb.ai/guides/core-types/models) captures structured parameters that control your application’s behavior, making it easy to find what parameters worked best. You can also use Weave a [`Model`](https://weave-docs.wandb.ai/guides/core-types/models) with serve, and [Evaluations](https://weave-docs.wandb.ai/guides/core-types/evaluations).

In the example below, you can experiment with `PersonExtractor`. Every time you change one of these, you'll get a new version of `PersonExtractor`.

In [ ]:
from typing import List, Iterable
from openai import AsyncOpenAI


# Define your desired output structure
class Person(BaseModel):
    person_name: str
    age: int


# Patch the OpenAI client
lm_client = instructor.from_openai(AsyncOpenAI())


class PersonExtractor(weave.Model):
    openai_model: str
    max_retries: int

    @weave.op()
    async def predict(self, text: str) -> List[Person]:
        model = await lm_client.chat.completions.create(
            model=self.openai_model,
            response_model=Iterable[Person],
            max_retries=self.max_retries,
            stream=True,
            messages=[
                {
                    "role": "system",
                    "content": "You are a perfect entity extraction system",
                },
                {
                    "role": "user",
                    "content": f"Extract `{text}`",
                },
            ],
        )
        return [m async for m in model]


model = PersonExtractor(openai_model="gpt-4", max_retries=2)
await model.predict("John is 30 years old")

## If you're running from a python script, uncomment and use the following code instead of `await`
# import asyncio
# asyncio.run(model.predict("John is 30 years old"))

| [![](https://weave-docs.wandb.ai/assets/images/instructor_weave_model-313d13f30307a5f2dad3063349e55c4f.png)](https://wandb.ai/geekyrakshit/quickstart-instructor/weave/calls) |
| --- |
| Tracing and versioning your calls using a [`Model`](https://weave-docs.wandb.ai/guides/core-types/models) |